# PREPROCESSING

In [78]:
# import all libraries
import pandas as pd

In [79]:
### import data
df = pd.read_csv('/home/sam/Desktop/Codes/DataScience/datasets/fake-news/train.csv')

In [176]:
### drop nan values
df = df.dropna()

In [177]:
### split x and Y
x = df.drop('label', axis=1)
y = df['label']

In [178]:
import tensorflow as tf

In [179]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM

In [180]:
### Vocabulary size
voc_size = 5000

In [181]:
# one_hot represention
message = x.copy()
message.reset_index(inplace = True)

In [182]:
import nltk
import re
from nltk.corpus import stopwords

In [183]:
# Dataset Preprocessing

In [184]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(message)):
    sample = re.sub('[^a-zA-Z]',' ', message['title'][i])
    sample = sample.lower()
    sample = sample.split()
    
    sample = [ps.stem(word) for word in sample if not word in stopwords.words('english')]
    sample = ' '.join(sample)
    corpus.append(sample)

In [136]:
onehot_repre = [one_hot(words, voc_size)for words in corpus]

In [137]:
sent_length = 20
embeded_docs = pad_sequences(onehot_repre, padding='pre', maxlen=sent_length)

In [196]:
# Creating model
embedding_vector_feature = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_feature, input_length = sent_length))
embedd_layer = Embedding(voc_size, embedding_vector_feature, input_length = sent_length)

model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer= 'adam', metrics = ['accuracy'])

In [191]:
import numpy as np
x_final = np.array(embeded_docs)
y_final = np.array(y)
x_final.shape, y_final.shape

((18285, 20), (18285,))

In [140]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size =0.3, random_state =42)

In [141]:
#finally Training
model.fit(x_train, y_train, validation_data=(x_test,y_test), epochs = 10, batch_size=64)

Train on 12799 samples, validate on 5486 samples
Epoch 1/10
12799/12799 [==============================] - 8s 588us/sample - loss: 0.3266 - acc: 0.8562 - val_loss: 0.1911 - val_acc: 0.9178
Epoch 2/10
12799/12799 [==============================] - 6s 507us/sample - loss: 0.1362 - acc: 0.9465 - val_loss: 0.1970 - val_acc: 0.9218
Epoch 3/10
12799/12799 [==============================] - 6s 505us/sample - loss: 0.0983 - acc: 0.9638 - val_loss: 0.2174 - val_acc: 0.9182
Epoch 4/10
12799/12799 [==============================] - 7s 508us/sample - loss: 0.0763 - acc: 0.9734 - val_loss: 0.2639 - val_acc: 0.9172
Epoch 5/10
12799/12799 [==============================] - 6s 505us/sample - loss: 0.0537 - acc: 0.9825 - val_loss: 0.2798 - val_acc: 0.9098
Epoch 6/10
12799/12799 [==============================] - 7s 513us/sample - loss: 0.0345 - acc: 0.9895 - val_loss: 0.3315 - val_acc: 0.9120
Epoch 7/10
12799/12799 [==============================] - 7s 514us/sample - loss: 0.0250 - acc: 0.9918 - val_lo

In [142]:
y_pred =model.predict_classes(x_test)

In [143]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[2809,  298],
       [ 183, 2196]])

In [166]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9123222748815166

In [174]:
v = x_test[20:30,:]

In [175]:
model.predict_classes(v)

array([[0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1]], dtype=int32)